In [1]:
import pandas as pd
samples = pd.read_csv('/Users/Surya/Downloads/screen-tsv-download.tsv', header=None, delimiter='\t')
samples.columns = ['cell_type','tissue']
leuk_samples = samples[samples['tissue'].str.contains('bone', case=False, na=False)]
print("There are", len(leuk_samples), "samples taken from a bone element/bone marrow")


There are 34 samples taken from a bone element/bone marrow


### Look
Look at big wig for open chromatin, can load it into IGV

locate potenital binding sites upstream of the promoter.

Identify potential binding sites, where it COULD happen; looking for consensus cis-regulatory elements identified upstream

If it's not present in AML, then maybe which cancers can we see it happening.

# Right Now
Let's try and get a scraping of protein atlas, query all of those cell lines, to make sure we got 'em all. Try with hyphen, no space-all text, replace hyphen with space, make sure that exclude chronic and not case sensitive

Let's try and get a visual understanding of binding sites and how we can approach implementing bigWig download from API

##  Creating a Comprehensive Profile of Available Leukemia Cell Lines in ENCODE

Acute Lymphoblastic Leukemia Cell Lines: 

https://www.cell.com/trends/immunology/abstract/S1471-4906(20)30307-0 

https://www.nature.com/articles/s41467-022-29224-5

Acute Myleoid Leukemia Cell lines: 

https://www.proteinatlas.org/humanproteome/cell+line/leukemia 

https://pmc.ncbi.nlm.nih.gov/articles/PMC10049680/table/ijms-24-05377-t001/ 



In [2]:
samples_ALL = ['DND-41','Jurkat','Loucy']
samples_AML = ['DND-41','HL-60', 'NB4', 'CMK']


## Scraping Protein Atlas for Cell Lines to Query for all Leukemia Samples for Verification of Above
https://www.proteinatlas.org/humanproteome/cell+line/leukemia

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
# URL of the Protein Atlas Leukemia page
url = "https://www.proteinatlas.org/humanproteome/cell+line/leukemia"

# Open the webpage
driver.get(url)

# Wait for the page to load (adjust as needed for specific elements)
driver.implicitly_wait(10)

# Get the page source after rendering
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Locate and extract the desired script
script_tags = soup.find_all('script')
for script in script_tags:
    if "var plot = $('#celline_prio_leukemia_LAML')" in script.text:
        full_line = script.text
        print("Extracted Script:")
        print(full_line)
        break

# Close the browser
driver.quit()


Extracted Script:

        var plot = $('#celline_prio_leukemia_LAML').scatterPlot([{"color":"lightgrey","data":[{"class":"cl938","name":"Kasumi-6","x":1.96068175909325,"y":0.834886071020256,"tooltip":"Kasumi-6<br>NES: 1.96<br>Corr: 0.83","url":""},{"class":"cl1095","name":"MOLM-6","x":1.90032457194471,"y":0.835674513933875,"tooltip":"MOLM-6<br>NES: 1.90<br>Corr: 0.84","url":""},{"class":"cl974","name":"KO52","x":1.98706550466945,"y":0.833892719414413,"tooltip":"KO52<br>NES: 1.99<br>Corr: 0.83","url":""},{"class":"cl1110","name":"MUTZ-3","x":1.88884309348927,"y":0.834852139753001,"tooltip":"MUTZ-3<br>NES: 1.89<br>Corr: 0.83","url":""},{"class":"cl694","name":"GDM-1","x":1.9503423496079,"y":0.823659272269025,"tooltip":"GDM-1<br>NES: 1.95<br>Corr: 0.82","url":""},{"class":"cl936","name":"Kasumi-1","x":1.86985160948779,"y":0.815727956855209,"tooltip":"Kasumi-1<br>NES: 1.87<br>Corr: 0.82","url":""},{"class":"cl1103","name":"Mono-Mac-1","x":1.8749012322619,"y":0.812513211118372,"tooltip":"M

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the Protein Atlas Leukemia page
url = "https://www.proteinatlas.org/humanproteome/cell+line/leukemia"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the script tag containing the desired identifier
    script_tags = soup.find_all('script')
    for script in script_tags:
        # Look for the specific identifier in the script content
        if "var plot = $('#celline_prio_leukemia_LAML')" in script.text:
            # Extract the full line containing the scatterPlot data
            full_line = re.search(r"var plot = \$\('#celline_prio_leukemia_LAML'\)\.scatterPlot\((.*?)\);", script.text, re.DOTALL)
            # if full_line:
            full_line_text = full_line.group(1)  # Extract the full JSON-like data
            print(full_line_text)  


[{"color":"lightgrey","data":[{"class":"cl938","name":"Kasumi-6","x":1.96068175909325,"y":0.834886071020256,"tooltip":"Kasumi-6<br>NES: 1.96<br>Corr: 0.83","url":""},{"class":"cl1095","name":"MOLM-6","x":1.90032457194471,"y":0.835674513933875,"tooltip":"MOLM-6<br>NES: 1.90<br>Corr: 0.84","url":""},{"class":"cl974","name":"KO52","x":1.98706550466945,"y":0.833892719414413,"tooltip":"KO52<br>NES: 1.99<br>Corr: 0.83","url":""},{"class":"cl1110","name":"MUTZ-3","x":1.88884309348927,"y":0.834852139753001,"tooltip":"MUTZ-3<br>NES: 1.89<br>Corr: 0.83","url":""},{"class":"cl694","name":"GDM-1","x":1.9503423496079,"y":0.823659272269025,"tooltip":"GDM-1<br>NES: 1.95<br>Corr: 0.82","url":""},{"class":"cl936","name":"Kasumi-1","x":1.86985160948779,"y":0.815727956855209,"tooltip":"Kasumi-1<br>NES: 1.87<br>Corr: 0.82","url":""},{"class":"cl1103","name":"Mono-Mac-1","x":1.8749012322619,"y":0.812513211118372,"tooltip":"Mono-Mac-1<br>NES: 1.87<br>Corr: 0.81","url":""},{"class":"cl1284","name":"OCI-AML-5

In [19]:
full_line_text

'[{"color":"lightgrey","data":[{"class":"cl938","name":"Kasumi-6","x":1.96068175909325,"y":0.834886071020256,"tooltip":"Kasumi-6<br>NES: 1.96<br>Corr: 0.83","url":""},{"class":"cl1095","name":"MOLM-6","x":1.90032457194471,"y":0.835674513933875,"tooltip":"MOLM-6<br>NES: 1.90<br>Corr: 0.84","url":""},{"class":"cl974","name":"KO52","x":1.98706550466945,"y":0.833892719414413,"tooltip":"KO52<br>NES: 1.99<br>Corr: 0.83","url":""},{"class":"cl1110","name":"MUTZ-3","x":1.88884309348927,"y":0.834852139753001,"tooltip":"MUTZ-3<br>NES: 1.89<br>Corr: 0.83","url":""},{"class":"cl694","name":"GDM-1","x":1.9503423496079,"y":0.823659272269025,"tooltip":"GDM-1<br>NES: 1.95<br>Corr: 0.82","url":""},{"class":"cl936","name":"Kasumi-1","x":1.86985160948779,"y":0.815727956855209,"tooltip":"Kasumi-1<br>NES: 1.87<br>Corr: 0.82","url":""},{"class":"cl1103","name":"Mono-Mac-1","x":1.8749012322619,"y":0.812513211118372,"tooltip":"Mono-Mac-1<br>NES: 1.87<br>Corr: 0.81","url":""},{"class":"cl1284","name":"OCI-AML-

In [8]:
parsed_js

{'type': 'Program',
 'body': [{'type': 'ExpressionStatement',
   'expression': {'type': 'SequenceExpression',
    'expressions': [{'type': 'ArrayExpression',
      'elements': [{'type': 'ObjectExpression',
        'properties': [{'type': 'Property',
          'key': {'type': 'Literal', 'value': 'color', 'raw': '"color"'},
          'computed': False,
          'value': {'type': 'Literal',
           'value': 'lightgrey',
           'raw': '"lightgrey"'},
          'kind': 'init',
          'method': False,
          'shorthand': False},
         {'type': 'Property',
          'key': {'type': 'Literal', 'value': 'data', 'raw': '"data"'},
          'computed': False,
          'value': {'type': 'ArrayExpression',
           'elements': [{'type': 'ObjectExpression',
             'properties': [{'type': 'Property',
               'key': {'type': 'Literal', 'value': 'class', 'raw': '"class"'},
               'computed': False,
               'value': {'type': 'Literal',
                'val

In [ ]:
full_line_text

'[{"color":"lightgrey","data":[{"class":"cl938","name":"Kasumi-6","x":1.96068175909325,"y":0.834886071020256,"tooltip":"Kasumi-6<br>NES: 1.96<br>Corr: 0.83","url":""},{"class":"cl1095","name":"MOLM-6","x":1.90032457194471,"y":0.835674513933875,"tooltip":"MOLM-6<br>NES: 1.90<br>Corr: 0.84","url":""},{"class":"cl974","name":"KO52","x":1.98706550466945,"y":0.833892719414413,"tooltip":"KO52<br>NES: 1.99<br>Corr: 0.83","url":""},{"class":"cl1110","name":"MUTZ-3","x":1.88884309348927,"y":0.834852139753001,"tooltip":"MUTZ-3<br>NES: 1.89<br>Corr: 0.83","url":""},{"class":"cl694","name":"GDM-1","x":1.9503423496079,"y":0.823659272269025,"tooltip":"GDM-1<br>NES: 1.95<br>Corr: 0.82","url":""},{"class":"cl936","name":"Kasumi-1","x":1.86985160948779,"y":0.815727956855209,"tooltip":"Kasumi-1<br>NES: 1.87<br>Corr: 0.82","url":""},{"class":"cl1103","name":"Mono-Mac-1","x":1.8749012322619,"y":0.812513211118372,"tooltip":"Mono-Mac-1<br>NES: 1.87<br>Corr: 0.81","url":""},{"class":"cl1284","name":"OCI-AML-

---------------------------------------------------------------------------------
### Attempt at Scraping Screen Directly --> Do BIGWIG download from API directly 
---------------------------------------------------------------------------------

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')     

# Path to the chromedriver, make sure chromedriver is in your PATH or provide the full path
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', options=options)

try:
    # Navigate to SCREEN
    url = "https://screen.encodeproject.org/search/?q=EIF6&assembly=GRCh38&uuid=985f9f7f-730a-49d4-8dbf-ee36eab1ab43"
    driver.get(url)
    time.sleep(5)  # Wait for the page to fully load

    # Search for consensus cis-regulatory elements
    search_box = driver.find_element(By.XPATH, "//input[@type='search']")
    search_box.send_keys("EIF6")
    search_box.send_keys(Keys.RETURN)
    time.sleep(5)  # Wait for search results

    # Extract consensus cis-regulatory elements information
    results = driver.find_elements(By.CLASS_NAME, "result-item")
    for result in results:
        element_text = result.text
        if "TP53" in element_text:
            print("Found TP53 binding site information:\n")
            print(element_text)

finally:
    # Close the WebDriver
    driver.quit()

# Note: Adjust the XPATHs and CLASS_NAME according to the actual HTML structure of the SCREEN page.
# The sleep commands are used to wait for the page to load, but it's better to use explicit waits.


TypeError: __init__() got an unexpected keyword argument 'executable_path'